# Stokes Flow and Permeability Tensor

In [1]:
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
import numpy as np
import matplotlib.pyplot as plt

## Generate a Layered Network

In order to see a network with a noticable difference in permeability values in each principle direction, we'll make a network consisting of two distinct layers then stitch them together.  The process of stitching is explained in more detail [here](../../tutorials/network/stitching_and_merging_networks_together.ipynb).

In [2]:
pn = op.network.Cubic(shape=[8, 8, 8], spacing=30e-6)
pn2 = op.network.Cubic(shape=[24, 24, 8], spacing=10e-6)

### Apply Labels to Each Network

In order to make it easier to find pore indices after stitching, it's helpful to apply unique and descriptive labels to the networks first. Labels are discussed detail [here](../../reference/data_management/using_and_creating_labels.ipynb).

In [3]:
pn['pore.coarse'] = True
pn['throat.coarse'] = True
pn2['pore.fine'] = True
pn2['throat.fine'] = True

Now we can stitch the two network together, but first we must translate the fine network along the z-axis so the networks are not overlapping.  Stiching is described in more detail [here](../../tutorials/network/stitching_and_merging_networks_together.ipynb).

In [4]:
pn2['pore.coords'] += [0, 0, 8*30e-6]
op.topotools.stitch(network=pn, donor=pn2, 
                    P_network=pn.pores('top'), 
                    P_donor=pn2.pores('bottom'))

Let's visualize the network to see if it stitched as expected. Quick plotting is described in more detail [here](../../getting_started/quick_plotting_networks.ipynb).

In [5]:
fig = op.topotools.plot_connections(pn)

AttributeError: module 'openpnm.topotools' has no attribute 'plot_connections'

## Create other objects needed for simulation
### Generate one Geometry for each layer

In [6]:
np.random.seed(0)
Ps = pn.pores('fine')
Ts = pn.throats('fine')
geo2 = op.geometry.SpheresAndCylinders(network=pn, pores=Ps, throats=Ts)
Ps = pn.pores('coarse')
Ts = pn.throats(['coarse', 'stitched'])
geo1 = op.geometry.SpheresAndCylinders(network=pn, pores=Ps, throats=Ts)

AttributeError: module 'openpnm' has no attribute 'geometry'

Let's visualize the pore sizes using histograms:

In [7]:
fig = plt.hist(geo1['pore.diameter'], bins=25, 
               density=True, edgecolor='k', alpha=0.5)
fig = plt.hist(geo2['pore.diameter'], bins=25, 
               density=True, edgecolor='k', alpha=0.5)

NameError: name 'geo1' is not defined

### Create a Phase and 2 Physics objects

In [8]:
air = op.phase.Air(network=pn)
phys1 = op.physics.Standard(network=pn, phase=air, geometry=geo1)
phys2 = op.physics.Standard(network=pn, phase=air, geometry=geo2)

ModuleNotFoundError: No module named 'thermo'

Let's visualize the hydraulic conductances using histograms.  Note that the values are logged, to better visualize the distributions:

In [9]:
fig = plt.hist(np.log10(phys1['throat.hydraulic_conductance']), 
               bins=25, density=True, edgecolor='k', alpha=0.5)
fig = plt.hist(np.log10(phys2['throat.hydraulic_conductance']), 
               bins=25, density=True, edgecolor='k', alpha=0.5)

NameError: name 'phys1' is not defined

## Run a StokesFlow simulation in perpendicular directions

In [10]:
sf_x = op.algorithms.StokesFlow(network=pn, phase=air)
Pin = 1.0
Pout = 0.0
sf_x.set_value_BC(pores=pn.pores('left'), values=1.0)
sf_x.set_value_BC(pores=pn.pores('right'), values=0.0)
sf_x.run()

NameError: name 'air' is not defined

Let's make sure the pressure field is applied in the correct direction:

In [11]:
fig = op.topotools.plot_coordinates(pn, color=sf_x['pore.pressure'], 
                                    size_by=pn['pore.diameter'], 
                                    markersize=50)

AttributeError: module 'openpnm.topotools' has no attribute 'plot_coordinates'

In [12]:
sf_z = op.algorithms.StokesFlow(network=pn, phase=air)
Pin = 1.0
Pout = 0.0
sf_z.set_value_BC(pores=pn.pores(['top', 'coarse'], mode='and'), values=Pin)
sf_z.set_value_BC(pores=pn.pores(['top', 'fine'], mode='and'), values=Pout)
sf_z.run()

NameError: name 'air' is not defined

Again, let's visualize the pressure field:

In [13]:
fig = op.topotools.plot_coordinates(pn, color=sf_z['pore.pressure'], 
                                    size_by=pn['pore.diameter'], 
                                    markersize=50)

AttributeError: module 'openpnm.topotools' has no attribute 'plot_coordinates'

The presure in the coarse network is essentially constant since it's much more permeable than the fine layer.

### Determine the Permeability Coefficient for each direction

In [14]:
L = 8 * 30e-6
A = (8 * 30e-6)**2 + (8 * 24)*(10e-6)**2
mu = air['pore.viscosity'][0]
Q = sf_x.rate(pores=pn.pores('left'), mode='group')
Kx = Q*L*mu/(A*(Pin - Pout))
print('The permeability coefficient is:', Kx)

NameError: name 'air' is not defined

In [15]:
A = (8 * 30e-6)**2
L = (8 * 30e-6) + (8 * 10e-6)
mu = air['pore.viscosity'][0]
Q = sf_z.rate(pores=pn.pores(['top', 'coarse'], mode='and'), mode='group')
Kz = Q*L*mu/(A*(Pin - Pout))
print('The permeability coefficient is:', Kz)

NameError: name 'air' is not defined

In [16]:
Kx/Kz

NameError: name 'Kx' is not defined

The permeability coefficient for the direction through the 'fine' layer is about 2x lower than the direction parallel to it.  